In [1]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [32]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [3]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [8]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[1.8010e+02, 1.1000e+00],
         [3.9023e+01, 3.8038e-02],
         [4.3018e+01, 1.6817e-01],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[4.1519e+02, 1.1000e+00],
         [2.2707e+02, 1.6000e-02],
         [3.3713e+02, 5.0000e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[6.0608e+02, 1.1000e+00],
         [5.3763e+01, 1.4590e-03],
         [8.6599e+01, 1.3517e-03],
         ...,
         [5.8843e+02, 1.6743e-03],
         [5.9699e+02, 1.3744e-03],
         [6.1143e+02, 1.5128e-03]],

        ...,

        [[5.6857e+02, 1.1000e+00],
         [9.5085e+01, 3.6179e-02],
         [9.7101e+01, 1.8033e-02],
         ...,
         [5.5034e+02, 2.8981e-03],
         [5.5056e+02, 3.6117e-01],
         [5.6857e+02, 5.1634e-03]],

        [[1.9509e+02, 1.1000e+00],
         [1.0904e+02, 3.

In [9]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 126, 256, 256])


In [10]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [11]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [12]:
tmp_original['precursor_mz']

tensor([180.1019, 415.1900, 606.0840, 204.0660, 494.3250, 258.0714, 339.1211,
        344.2230, 524.3720, 568.5670, 195.0877, 161.1172])

In [13]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+'],)

In [14]:
tmp_original['mol_freq']

tensor([ 32.,  18.,  19.,   6., 125.,  12.,   6.,   5., 200.,  30., 252.,   3.])

In [15]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [16]:
sum(tmp_original['labels'])

tensor(12)

In [17]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1536, 1792, 2048, 2304, 2430, 2686])

In [18]:
len(tmp_original['labels'])

2942

In [19]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 126, 256, 256])

In [20]:
sum(tmp_original['batch_ptr'])

tensor(2942)

In [21]:
len(tmp_original['candidates'])

2942

In [22]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [23]:
len(tmp_original['candidates_smiles'])

2942

# MSnRetrieval

In [24]:
from massspecgym.featurize import SpectrumFeaturizer

In [35]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [36]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [37]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [38]:
# file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/min_sample_trees.mgf"


In [ ]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 16476
MSnRetrievalDataset length: 16476
Precomputing candidate-side transformations using multiprocessing with chunking...
Using 8 processes for precomputation.


Precomputing in chunks:  24%|██▎       | 4/17 [08:22<28:14, 130.35s/it]

In [30]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [31]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 82
Val dataset size: 6


In [32]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

[PROFILE] __getitem__ took 0.003323 seconds
[PROFILE] __getitem__ took 0.001279 seconds
[PROFILE] collate_fn took 0.028322 seconds
{'spec': DataBatch(x=[141, 1039], edge_index=[2, 258], batch=[141], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([540.2440, 730.2917, 393.1333, 435.1286, 401.1959, 334.1550, 496.2177,
        340.1543, 644.3065, 401.1595, 397.1646, 399.1438]), 'adduct': ['[M+NH4]+', '[M+NH4]+', '[M+H-H2O]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+NH4]+', '[M+H]+', '[M+NH4]+', '[M+H]+', '[M+H]+', '[M+H-H2O]+'], 'identifier': ['0000433_0000000', '0000457_0000000', '0000131_0000000', '0000423_0000000', '0000272_0000000', '0000174_0000000', '0000285_0000000', '0000264_0000000', '0000397_0000000', '0000546_0000000', '0000187_0000000', '0

In [33]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256,  35, 256, 256, 256, 256, 256, 256, 146, 256, 256, 256],
       dtype=torch.int32)


In [34]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

Data types in tmp_original:
  spec: <class 'torch.Tensor'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  identifier: <class 'list'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>
Data types in tmp_msn:
  spec: <class 'abc.DataBatch'>
  mol: <class 'torch.Tensor'>
  precursor_mz: <class 'torch.Tensor'>
  adduct: <class 'list'>
  identifier: <class 'list'>
  mol_freq: <class 'torch.Tensor'>
  smiles: <class 'list'>
  candidates: <class 'torch.Tensor'>
  labels: <class 'torch.Tensor'>
  batch_ptr: <class 'torch.Tensor'>
  candidates_smiles: <class 'list'>


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [35]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [36]:
tmp_msn['spec'] 

DataBatch(x=[141, 1039], edge_index=[2, 258], batch=[141], ptr=[13])

In [37]:
tmp_msn['precursor_mz']

tensor([540.2440, 730.2917, 393.1333, 435.1286, 401.1959, 334.1550, 496.2177,
        340.1543, 644.3065, 401.1595, 397.1646, 399.1438])

In [38]:
tmp_msn['mol_freq']

tensor([2., 1., 2., 1., 2., 1., 1., 1., 1., 4., 1., 3.])

In [39]:
tmp_msn['identifier']

['0000433_0000000',
 '0000457_0000000',
 '0000131_0000000',
 '0000423_0000000',
 '0000272_0000000',
 '0000174_0000000',
 '0000285_0000000',
 '0000264_0000000',
 '0000397_0000000',
 '0000546_0000000',
 '0000187_0000000',
 '0000059_0000000']

In [40]:
tmp_msn['adduct']

['[M+NH4]+',
 '[M+NH4]+',
 '[M+H-H2O]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+H]+',
 '[M+NH4]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H-H2O]+']

In [41]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [42]:
tmp_msn['smiles']

['COC1=C(C=C2C(C(C(CC2=C1)CO)CO[C@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O)O)O)C4=CC(=C(C=C4)O)OC)O',
 'COC1=CC(=CC(=C1O)OC)[C@H]2C3CO[C@@H](C3CO2)C4=CC(=C(C(=C4)OC)O[C@H]5[C@@H]([C@H]([C@@H]([C@H](O5)CO)O)O)O[C@H]6[C@@H]([C@](CO6)(CO)O)O)OC',
 'CC(=C)[C@@H]1CC2=C(O1)C=CC3=C2O[C@H]4COC5=CC(=C(C=C5[C@]4(C3=O)O)OC)OC',
 'C1[C@H](OC2=C(C1=O)C(=C(C(=C2)O)[C@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O)O)O)O)C4=CC=C(C=C4)O',
 'CC1CC2=CC3=C(C(=C2C4=C(C(=C(C=C4CC1C)OC)OC)OC)OC)OCO3',
 'C1=CC=C(C=C1)CC2C(=O)NC(C(=O)N2)CC3=CNC4=CC=CC=C43',
 'C1[C@H]([C@@H]([C@H]([C@@H](O1)O[C@@H](CCC2=CC(=C(C=C2)O)O)CC(=O)CCC3=CC(=C(C=C3)O)O)O)O)O',
 'COC1=C(C=C(C=C1)CC2=NC=CC3=CC(=C(C=C32)OC)OC)OC',
 'C/C=C(\\C)/C(=O)OC1C(C(C2(C3CCC4(C(OC(=O)CC4=C3CC1(C2=O)O)C5=COC=C5)C)C)C(C(=O)OC)OC(=O)C)(C)C',
 'CC1=C2C(=CC=C1)OC(=O)C3(C2=O)C(C(=O)CC3(C)C=C)C(CO)COC(=O)C',
 'CC(=CCC1=C(C2=C(C=C1O)OC3=C(C2=O)C(=C(C(=C3)O)O)CC=C(C)C)O)C',
 'C/C=C(\\C)/C(=O)OC(COC1=C2C(=C(C3=C1OC(=O)C=C3)OC)C=CO2)C(C)(C)O']

In [43]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [44]:
sum(tmp_msn['labels'])

tensor(12)

In [45]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  291,  547,  803, 1059, 1315, 1571, 1827, 1973, 2229, 2485])

In [46]:
len(tmp_msn['labels'])

2741

In [47]:
tmp_msn['batch_ptr']

tensor([256,  35, 256, 256, 256, 256, 256, 256, 146, 256, 256, 256],
       dtype=torch.int32)

In [48]:
sum(tmp_msn['batch_ptr'])

tensor(2741, dtype=torch.int32)

In [49]:
len(tmp_msn['candidates'])

2741

In [50]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [51]:
tmp_msn.keys() == tmp_original.keys()

True

In [52]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [53]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [54]:
tmp_original['batch_ptr'].dtype

torch.int64

In [55]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [56]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2741, dtype=torch.int32), 2741)

In [57]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(3055), 3055)

In [58]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [59]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [60]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [61]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [62]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [63]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [64]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [65]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [66]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [67]:
len(tmp_msn['candidates_smiles'])

2741